In [ ]:
import pandas as pd
import requests
import json
import os
import time

# ================= 配置区域 =================

# 1. 接口配置
API_URL = "https://www.dacoknow.com/gpt/digBeijingTag/queryCommon"
USER_KEY = "milvus_251210"
TABLE_NAME = "milvus_251210"

# 【注意】虽然接口支持 topK: 100，但为了 Excel 可读性，
# 这里默认只提取前 3 条最相似的结果写入文件。
# 如果你需要更多，请修改 SAVE_TOP_K。
REQUEST_TOP_K = 20  # 请求接口时获取的数量
SAVE_TOP_K = 3      # 存入 Excel 的数量

# 2. 文件路径配置
# 自动获取当前工作目录
try:
    CURRENT_DIR = os.path.dirname(os.path.abspath(__file__))
except NameError:
    CURRENT_DIR = os.getcwd() # 兼容 Jupyter Notebook

print(f"当前工作目录: {CURRENT_DIR}")

# 输入文件 (确保这个文件在当前目录下)
INPUT_FILE = os.path.join(CURRENT_DIR, "question_expanded_result_v2.xlsx")
# 输出文件
OUTPUT_FILE = os.path.join(CURRENT_DIR, "api_retrieval_result_final.xlsx")

# ===========================================

def call_search_api(query_text):
    """
    调用远程接口进行搜索并解析特定格式的返回结果
    """
    if not query_text or pd.isna(query_text) or str(query_text).strip() == "":
        return []

    # 1. 构造请求参数 (严格按照你提供的格式)
    payload = {
        "userKey": USER_KEY,
        "tableName": TABLE_NAME,
        "qas": str(query_text),
        "topK": REQUEST_TOP_K
    }
    
    headers = {
        "Content-Type": "application/json"
    }

    try:
        # 2. 发送 POST 请求
        # verify=False 是为了防止某些自签名 SSL 证书报错，如果报错 SSL 相关错误请保留
        response = requests.post(API_URL, json=payload, headers=headers, timeout=30)
        
        if response.status_code != 200:
            return [f"HTTP错误: {response.status_code}"]
        
        result_json = response.json()
        
        # 3. 解析返回结果
        # 根据你提供的样例: {"success":true, "code":200, "result": [...]}
        if not result_json.get("success") and result_json.get("code") != 200:
             return [f"接口返回错误: {result_json.get('message', '未知错误')}"]

        data_list = result_json.get("result", [])
        
        retrieved_items = []
        
        if isinstance(data_list, list):
            # 只取前 SAVE_TOP_K 条写入 Excel
            for item in data_list[:SAVE_TOP_K]:
                # 提取字段
                # 优先取 'insertContent'，如果没有则取 '切片'
                content = item.get("insertContent") or item.get("切片") or ""
                score = item.get("distance")
                filename = item.get("文件名", "未知来源")
                
                # 格式化输出内容
                formatted_str = (
                    f"【距离: {score}】\n"
                    f"【来源: {filename}】\n"
                    f"{str(content)[:200]}..." # 截取前200字防止单元格爆掉
                )
                retrieved_items.append(formatted_str)
        else:
            retrieved_items.append(f"返回数据格式异常: {str(result_json)[:100]}")

        return retrieved_items

    except Exception as e:
        print(f"❌ 请求异常: {e}")
        return [f"Error: {e}"]

def main():
    print("--- 开始 API 检索对比任务 ---")

    # 1. 读取 Excel
    print(f"正在读取文件: {INPUT_FILE}")
    if not os.path.exists(INPUT_FILE):
        print(f"❌ 文件不存在: {INPUT_FILE}")
        return
    
    try:
        df = pd.read_excel(INPUT_FILE)
    except Exception as e:
        print(f"❌ 读取Excel失败: {e}")
        return
    
    # 2. 检查列名
    required_cols = ['方案1_单句扩写', '方案2_潜在追问']
    for col in required_cols:
        if col not in df.columns:
            print(f"❌ Excel 中缺少列: {col}，请检查上一步生成的文件。")
            print(f"当前列名: {list(df.columns)}")
            return

    # 准备存储结果
    scheme1_results = []
    scheme2_results = []

    total = len(df)
    print(f"共 {total} 条数据，开始调用接口...")

    # 3. 循环处理
    for index, row in df.iterrows():
        # 打印进度
        print(f"[{index+1}/{total}] 正在检索...", end="\r")
        
        # --- 方案 1 检索 ---
        q1 = row['方案1_单句扩写']
        res1 = call_search_api(q1)
        # 用换行符分隔多条召回结果
        scheme1_results.append("\n\n------------------------\n\n".join(res1))

        # --- 方案 2 检索 ---
        q2 = row['方案2_潜在追问']
        res2 = call_search_api(q2)
        scheme2_results.append("\n\n------------------------\n\n".join(res2))
        
        # 避免请求过快
        time.sleep(0.1) 

    print(f"\n✅ 所有请求完成！正在保存结果...")

    # 4. 保存结果
    df['方案1_API召回结果'] = scheme1_results
    df['方案2_API召回结果'] = scheme2_results

    try:
        df.to_excel(OUTPUT_FILE, index=False)
        print(f"🎉 成功！结果已保存至: {OUTPUT_FILE}")
    except PermissionError:
        print("❌ 保存失败：请关闭正在打开的 Excel 文件后重试。")

if __name__ == "__main__":
    main()

当前工作目录: d:\jupyter
--- 开始 API 检索对比任务 ---
正在读取文件: d:\jupyter\question_expanded_result_v2.xlsx
共 83 条数据，开始调用接口...
[83/83] 正在检索...
✅ 所有请求完成！正在保存结果...
🎉 成功！结果已保存至: d:\jupyter\api_retrieval_result_final.xlsx
